In [4]:
# Structured Inverse Paraphraser
# Very similar to DVP; except Dataset change

In [108]:
# ------------- IMP -------------
# Run for dataset one-by-one in different kaggle account
# While loading datasets make sure that label file has only 1 label
# Remove +1 while calculating the average train loss from denominator
# Remove global_step = 3 from saving the last model state
# set limit_examples to None while creating custom dataset class

In [59]:
# Desired Imports
import torch
import tqdm
from tqdm import trange
from transformers import (AdamW, GPT2Config, GPT2LMHeadModel, GPT2Tokenizer, get_linear_schedule_with_warmup)
from torch.utils.data import Dataset
import numpy as np
from collections import defaultdict
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
import os
import shutil
import subprocess
import json
import torch.nn.utils as F
from transformers import WEIGHTS_NAME

In [74]:
# Required directories
invp_paraphrase_model_chkpts_dir =  "/content/drive/MyDrive/IRE/INVP"
final_invp_paraphrase_model_dir = "/content/drive/MyDrive/IRE/INVP/final_INVP"
input_output_dir = "/content/drive/MyDrive/IRE/PSUEDO_PARALLEL" # from DVP

In [73]:
# Dataset for inverse paraphrasing
database_config = {
    "shakespeare" : ("shakespeare", "original"), # (dir_name, style_label)
    "poetry" : ("poetry", "romantic_poetry"),
    "bible" : ("bible", "bible")
}

dataset_to_use = "poetry"

In [75]:
# Arguments needed for training
DIR_NAME = 0
STYLE_LABEL = 1
dataset_to_run = database_config[dataset_to_use][DIR_NAME]
style_type = database_config[dataset_to_use][STYLE_LABEL]

args_dir = {
  "save_steps" : 20, # can be changed
  "num_epochs" : 3,
  "gradient_accumulation_steps" : 2,
  "adam_epsilon" : 1e-8,
  "warmup_steps" : 0,
  "learning_rt" : 5e-5,
  "max_grad_norm" : 1.0,
  "data_dir" : input_output_dir + "/" + dataset_to_run + "/BPE",
  "model_type" : "gpt2",
  "model_name" : "gpt2",  # set to gtp2-large
  "train_batch_size" : 5,
  "eval_batch_size" : 5,
  "extra_embedding_dim" : 768,
  "global_dense_feature_list" : None # in file it will be saved with the value null; while reading take care of this thing
}

model_type = args_dir["model_type"]
model_name = args_dir["model_name"]
data_dir = args_dir["data_dir"]
save_steps = args_dir["save_steps"]
num_epochs = args_dir["num_epochs"]
gradient_accumulation_steps = args_dir["gradient_accumulation_steps"]
adam_epsilon = args_dir["adam_epsilon"]
warmup_steps = args_dir["warmup_steps"]
train_batch_size = args_dir["train_batch_size"]
eval_batch_size = args_dir["eval_batch_size"]
learning_rt = args_dir["learning_rt"]
extra_embedding_dim = args_dir["extra_embedding_dim"] # Size of linear layer used for projecting extra embeddings.
global_dense_feature_list = args_dir["global_dense_feature_list"]
max_grad_norm = args_dir["max_grad_norm"]

In [76]:
# Choose device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpus = torch.cuda.device_count()

print(device, n_gpus)

cpu 0


In [14]:
!pip install transformers

In [77]:
# Initialize model classes variables
MODEL_CLASSES = {
    'gpt2': (GPT2Config, GPT2LMHeadModel, GPT2Tokenizer),
}
config_class, model_class, tokenizer_class = MODEL_CLASSES[model_type]

print("GPT2 Config class- ", config_class)
print("GPT2 Model class- ", model_class)
print("GPT2 Tokenizer class- ", tokenizer_class)

GPT2 Config class-  <class 'transformers.models.gpt2.configuration_gpt2.GPT2Config'>
GPT2 Model class-  <class 'transformers.models.gpt2.modeling_gpt2.GPT2LMHeadModel'>
GPT2 Tokenizer class-  <class 'transformers.models.gpt2.tokenization_gpt2.GPT2Tokenizer'>


In [78]:
# Init config
config = config_class.from_pretrained(model_name)
print("GPT2Config loaded")

# Init model
model = model_class.from_pretrained(model_name, config = config)
print("GPT2LMHeadModel loaded")

# Init tokenizer
tokenizer = tokenizer_class.from_pretrained(model_name,do_lower_case = False)
print("GPT2Tokenizer loaded")

GPT2Config loaded
GPT2LMHeadModel loaded
GPT2Tokenizer loaded


In [79]:
# adding extra_dimension to config --- No need in DVP ---- Can be removed later
config.extra_embedding_dim = extra_embedding_dim # don't know why we are using it; -- explore it

In [80]:
# Add special tokens to tokenizer
SPECIAL_TOKENS = {
    "additional_special_tokens": ["<segment_1>", "<segment_2>"],
    "pad_token": "<pad>",
    "bos_token": "<bos>",
    "eos_token": "<eos>"
}
tokenizer.add_special_tokens(SPECIAL_TOKENS)
print("Special Tokens addded to tokenizer")

print("Total tokens- ", len(tokenizer))

Special Tokens addded to tokenizer
Total tokens-  50262


In [81]:
# resize token embedding matrix to take care of special tokens added
model.resize_token_embeddings(len(tokenizer)) # each token of size-> 1280(gpt2-large), 768(gpt2)

Embedding(50262, 768)

In [82]:
# move model to device
model.to(device)

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50262, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50262, bias=False)
)

In [83]:
# Init configs

MAX_PARAPHRASE_LEN = 100

# mainly to handle input
CONFIG = {
    "max_prefix_len": int(MAX_PARAPHRASE_LEN / 2),
    "max_suffix_len": int(MAX_PARAPHRASE_LEN / 2)
}

In [84]:
# Preprocess input from paranmt
def preprocess(exp, tokenizer, config):
  MASK_TOKEN_ID = -100

  max_prefix_len = config["max_prefix_len"]
  max_suffix_len = config["max_suffix_len"]

  sent1 = np.array(exp["prefix_sent"])
  sent2 = np.array(exp["actual_sent"])

  # truncate
  if(len(sent1) > max_prefix_len):
    sent1 = sent1[:max_prefix_len]

  if(len(sent2) > max_suffix_len):
    sent2 = sent2[:max_suffix_len]

  # add padding; left padding to prefix and right padding to suffix
  count_pad_tokens_prefix = max_prefix_len - len(sent1)
  sent1 = np.pad(sent1, (count_pad_tokens_prefix, 0), constant_values = tokenizer.pad_token_id)

  # add <eos> to suffix
  sent2 = np.append(sent2, tokenizer.eos_token_id)

  count_pad_tokens_suffix = (max_suffix_len + 1) - len(sent2)
  sent2 = np.pad(sent2, (0, count_pad_tokens_suffix), constant_values = tokenizer.pad_token_id)

  # sentence to input gpt2
  sentence_to_input_gpt2 = np.concatenate([sent1, [tokenizer.bos_token_id], sent2]).astype(np.int64) # [sent1, <bos> sent2]

  # label/gt to predict; -100 used for masking that input (in ground truth only)
  gt = np.concatenate([
      [MASK_TOKEN_ID for _ in sent1],
      [MASK_TOKEN_ID],
      [val if val != tokenizer.pad_token_id else MASK_TOKEN_ID for val in sent2]
  ]).astype(np.int64)

  # segment
  segment = np.concatenate([
      [tokenizer.additional_special_tokens_ids[0] for _ in sent1],
      [tokenizer.additional_special_tokens_ids[1]],
      [tokenizer.additional_special_tokens_ids[1] for _ in sent2]
  ]).astype(np.int64)

  exp["prefix_sent"] = sent1
  exp["suffix_sent"] = sent2

  exp["input"] = sentence_to_input_gpt2
  exp["label"] = gt
  exp["segment"] = segment

  return exp

In [85]:
# INVP dataset
class Inverse_Paraphraser_Dataset(Dataset):
    def __init__(self, config, tokenizer, style_type, limit_examples = None, evaluate = False, split_type = "train"):
      self.config = config
      self.examples = []
      self.label_dict, self.reverse_label_dict = {style_type : 0}, {0 : style_type}

      file_to_read = data_dir + "/" + split_type + ".input0.bpe"
      with open(file_to_read) as f:
        author_input_data = f.read().strip().split("\n")

      file_to_read = data_dir + "/" + split_type + ".label"
      with open(file_to_read) as f:
        suffix_styles = f.read().strip().split("\n")

      file_to_read = data_dir + "/" + split_type + ".paraphrase_250_input0.bpe"
      with open(file_to_read) as f:
        prefix_data = f.read().strip().split("\n")

      print("Actual data(BPE)- ", author_input_data[0])
      print("Paraphrased data(BPE)- ", prefix_data[0])
      print("Suffix style- ", suffix_styles[0])

      self.examples = []
      for i, (act_sent, suffix_style) in tqdm.tqdm(enumerate(zip(author_input_data, suffix_styles)), total=len(author_input_data)):
        self.examples.append({
            "prefix_sent": np.array([int(x) for x in prefix_data[i].split()], dtype = np.int32),  # from DVP
            "actual_sent": np.array([int(x) for x in act_sent.split()], dtype = np.int32),        # from actual
            "suffix_style": self.label_dict[suffix_style],
        })

      if(limit_examples != None):
        self.examples = self.examples[:limit_examples]

      print(self.examples[0])

      for eg in self.examples:
        eg["original_style"] = eg["suffix_style"]

      # perform same preprocess as that of DVP
      self.examples = [preprocess(exp, tokenizer, config) for exp in self.examples]


    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
      sentence = self.examples[idx]["input"]
      label = self.examples[idx]["label"]
      segment = self.examples[idx]["segment"]

      return {
          "sample_number": idx,
          "sentence": torch.tensor(sentence),
          "label": torch.tensor(label),
          "segment": torch.tensor(segment)
      }

In [86]:
# create DVP dataset
train_dataset = Inverse_Paraphraser_Dataset(CONFIG,
                                            tokenizer,
                                            style_type,
                                            limit_examples = 10,
                                            evaluate = False, split_type = "train")
print("\n\n INVP Dataset created")

Actual data(BPE)-  37 533 4053 11 1468 32586 338 30942 18209 11
Paraphrased data(BPE)-  11274 12 16390 11 262 30942 10747 286 32586 11
Suffix style-  romantic_poetry


100%|██████████| 26880/26880 [00:01<00:00, 24941.79it/s]


{'prefix_sent': array([11274,    12, 16390,    11,   262, 30942, 10747,   286, 32586,
          11], dtype=int32), 'actual_sent': array([   37,   533,  4053,    11,  1468, 32586,   338, 30942, 18209,
          11], dtype=int32), 'suffix_style': 0}


 INVP Dataset created


In [87]:
# create dataloader
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler = train_sampler, batch_size = train_batch_size)

print("INVP train dataloader created")

INVP train dataloader created


In [38]:
next(iter(train_dataloader))

{'sample_number': tensor([3, 8, 0, 7, 2]),
 'sentence': tensor([[50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259,    40,  1101,  1654,   345,   821,   826,   546,   326,
          50260,  5779,   531,    13, 50261, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259],
         [50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259, 50259,
          50259, 50259, 50259, 50259, 50

In [ ]:
# Now follow the same as done in DVP

In [88]:
# Total steps needed
t_total = len(train_dataloader) // gradient_accumulation_steps * num_epochs

# setting up the optimizer & learning rate schedulers
no_decay = ['bias', 'LayerNorm.weight', 'layer_norm.weight']
grouped_parameters = [
    {
        'params': [p for n, p in model.named_parameters()],
        'weight_decay': 0.0
    }
]

optimizer = AdamW(grouped_parameters, lr = float(learning_rt), eps = adam_epsilon)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps = warmup_steps, num_training_steps = t_total)

print("Adam Optimizer and learning rate scheduler instantiated")

Adam Optimizer and learning rate scheduler instantiated


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [89]:
# Training meta Information
print("Num of examples- ", len(train_dataset))
print("Num of epochs- ", num_epochs)
print("Batch size- ", train_batch_size)
print("Gradient acculmulation steps- ", gradient_accumulation_steps)
print("Total optimization steps- ", t_total)

Num of examples-  10
Num of epochs-  3
Batch size-  5
Gradient acculmulation steps-  2
Total optimization steps-  3


In [90]:
# zero out all the gradients
model.zero_grad()

In [91]:
# Fn to save checkpoints
def save_model(model, tokenizer, chkpt_dir, global_step, args_dir):
  if not os.path.exists(chkpt_dir):
    os.makedirs(chkpt_dir)
  print("Directory created for new checkpt to save")

  model.save_pretrained(chkpt_dir)
  tokenizer.save_pretrained(chkpt_dir)
  print("Model and tokenizer saved")

  # save training arguments also
  with open(chkpt_dir + "/my_args.json", "w") as json_file:
    json.dump(args_dir, json_file)
  print("Training arguments saved")

  with open(os.path.join(chkpt_dir, "global_step.txt"), "w") as f:
    f.write(str(global_step) + "\n")
  print("Global step file saved")

  print("Checkpint saving process done..")

In [92]:
from tqdm import tqdm

global_step = 0
train_loss_val = 0.0
chkpts_dir_name = []

# start training
train_iterator = trange(int(num_epochs), desc = "Epoch")
for epoch in train_iterator:
    epoch_iterator = tqdm(train_dataloader)

    for batch_idx, batch in enumerate(epoch_iterator):
      sentences = batch["sentence"].to(device)
      labels = batch["label"].to(device)
      segments = batch["segment"].to(device)
      model.train()

      outputs = model(input_ids=sentences, token_type_ids = segments, labels = labels)
      # print("Got logits and loss")

      loss = outputs.loss
      loss = loss / gradient_accumulation_steps
      train_loss_val += loss.item()

      loss.backward()

      if (((batch_idx + 1) % gradient_accumulation_steps) == 0):
        # print("Moved 1 step")
        F.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()

        model.zero_grad()
        global_step += 1

        if (global_step % save_steps == 0):
          # save checkpoint here
          print("Saving new checkpoint")
          chkpt_dir = invp_paraphrase_model_chkpts_dir + "/" + dataset_to_run + "/invp_chkpt_"+str(global_step)
          chkpts_dir_name.append("invp_chkpt_"+str(global_step))

          save_model(model, tokenizer, chkpt_dir, global_step, args_dir)
      # break

Epoch: 100%|██████████| 3/3 [00:57<00:00, 19.02s/it]


In [93]:
# Average train_loss per step
global_step, tr_loss = global_step, train_loss_val / (global_step + 1) # +1 only while testing
print("Final Global step- ", global_step)
print("Average training loss per step- ", tr_loss)

Final Global step-  0
Average training loss per step-  111.62308502197266


In [94]:
# save the last model also
global_step = 3  # remove this also; fetch the last global step value
chkpt_dir = invp_paraphrase_model_chkpts_dir + "/" + dataset_to_run + "/invp_chkpt_"+str(global_step)
chkpts_dir_name.append("invp_chkpt_"+str(global_step))

save_model(model, tokenizer, chkpt_dir, global_step, args_dir)

print("Last model state saved")

Directory created for new checkpt to save
Model and tokenizer saved
Training arguments saved
Global step file saved
Checkpint saving process done..
Last model state saved


In [95]:
# Till now,
# INVP trained on chosen dataset -> checkpoints saved -> Last model state saved
print("Checkpoints saved with the name- ", chkpts_dir_name)

Checkpoints saved with the name-  ['invp_chkpt_3']


In [96]:
# Start Evaluation
print("Starting Evaluation of INVP on dev data based on perplexity")

Starting Evaluation of INVP on dev data based on perplexity


In [99]:
# Get validation dataset and dataloader
import tqdm
val_dataset = Inverse_Paraphraser_Dataset(CONFIG,
                                          tokenizer,
                                          style_type,
                                          limit_examples = 10,
                                          evaluate = True, split_type = "dev")

val_sampler = SequentialSampler(val_dataset)
val_dataloader = DataLoader(val_dataset, sampler = val_sampler, batch_size = eval_batch_size)

print("Validation dataset and dataloader created")

Actual data(BPE)-  37 533 4053 11 1468 32586 338 30942 18209 11
Paraphrased data(BPE)-  11274 12 16390 11 262 30942 10747 286 32586 11
Suffix style-  romantic_poetry


100%|██████████| 26880/26880 [00:00<00:00, 58435.93it/s]

{'prefix_sent': array([11274,    12, 16390,    11,   262, 30942, 10747,   286, 32586,
          11], dtype=int32), 'actual_sent': array([   37,   533,  4053,    11,  1468, 32586,   338, 30942, 18209,
          11], dtype=int32), 'suffix_style': 0}
Validation dataset and dataloader created


In [100]:
# Validation meta Information
print("Num of examples- ", len(val_dataset))
print("Batch size- ", eval_batch_size)

Num of examples-  10
Batch size-  5


In [102]:
# Fn to evaluate on a DVP checkpoint
def evaluate(model, tokenizer, chkpt_dir_name, val_dataloader):
  val_loss = 0.0
  model.eval()

  for i, batch in enumerate(val_dataloader):
    sentences = batch["sentence"].to(device)
    labels = batch["label"].to(device)
    segments = batch["segment"].to(device)

    with torch.no_grad():
      op = model(input_ids=sentences, token_type_ids=segments, labels=labels)
      loss_val = op.loss.item()

    val_loss += loss_val
    # break

  avg_val_loss = val_loss / (i + 1) # per batch average loss
  perplexity = torch.exp(torch.tensor(avg_val_loss)) # perplexity of exp(avg_loss)

  return perplexity

In [103]:
chkpts_dir_name

['invp_chkpt_3']

In [104]:
# Start evaluating the checkpoints on dev data and using perplexity as a measure
perplexity_list = []

for chkpt_name in chkpts_dir_name:
  # load chkpoint
  chkpt_to_load = invp_paraphrase_model_chkpts_dir + "/" + dataset_to_run + "/" + chkpt_name
  model = model_class.from_pretrained(chkpt_to_load)
  tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)
  model.to(device)
  print("Checkpoint- " + chkpt_name + " loaded")

  # evaluate loaded
  print("Evaluating on loaded checkpoint")
  perplexity = evaluate(model, tokenizer, chkpt_dir, val_dataloader)
  perplexity_list.append((chkpt_name, perplexity))

print("INVP evaluated on all the saved checkpoints")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Checkpoint- invp_chkpt_3 loaded
Evaluating on loaded checkpoint
INVP evaluated on all the saved checkpoints


In [105]:
# Sort perplexity list in increasing order to get best model
perplexity_list.sort(key=lambda x: x[1].item())
top_chkpt_name = perplexity_list[0][0]

print("Top performing checkpoint is- ", top_chkpt_name)

Top performing checkpoint is-  invp_chkpt_3


In [ ]:
# Evaluation on dev data done

In [106]:
# move top performing checkpoint to other final_paraphrase_model dir
copy_cmd = "cp {}/* {}".format(invp_paraphrase_model_chkpts_dir + "/" + dataset_to_run + "/" + top_chkpt_name,
                               final_invp_paraphrase_model_dir + "/" + dataset_to_run)
print(copy_cmd)

# do copy to some other location
subprocess.check_output(copy_cmd, shell=True)
print("Copied successfully..!!")

cp /content/drive/MyDrive/IRE/INVP/poetry/invp_chkpt_3/* /content/drive/MyDrive/IRE/INVP/final_INVP/poetry
Copied successfully..!!


In [107]:
# just to verify that copied model is loading correctly or not
chkpt_to_load = final_invp_paraphrase_model_dir + "/" + dataset_to_run
model = model_class.from_pretrained(chkpt_to_load)
tokenizer = tokenizer_class.from_pretrained(chkpt_to_load, do_lower_case = True)

print("Model loaded successfully..!!")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Model loaded successfully..!!


In [ ]:
# Finish Inverse Paraphraser